In [32]:
from keras.applications import VGG16
 
vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [33]:
train_dir = 'img/train'
validation_dir = 'img/validation'
 
nTrain = 570
nVal = 150

In [21]:
!ls img_tiger

tiger.10.jpg  tiger.199.jpg tiger.213.jpg tiger.229.jpg tiger.30.jpg
tiger.11.jpg  tiger.20.jpg  tiger.214.jpg tiger.23.jpg  tiger.31.jpg
tiger.12.jpg  tiger.200.jpg tiger.215.jpg tiger.230.jpg tiger.32.jpg
tiger.13.jpg  tiger.201.jpg tiger.217.jpg tiger.235.jpg tiger.4.jpg
tiger.14.jpg  tiger.203.jpg tiger.218.jpg tiger.24.jpg  tiger.5.jpg
tiger.15.jpg  tiger.204.jpg tiger.219.jpg tiger.245.jpg tiger.6.jpg
tiger.16.jpg  tiger.205.jpg tiger.22.jpg  tiger.246.jpg tiger.7.jpg
tiger.17.jpg  tiger.206.jpg tiger.220.jpg tiger.247.jpg tiger.8.jpg
tiger.18.jpg  tiger.207.jpg tiger.222.jpg tiger.248.jpg tiger.9.jpg
tiger.19.jpg  tiger.208.jpg tiger.223.jpg tiger.249.jpg train
tiger.194.jpg tiger.209.jpg tiger.224.jpg tiger.25.jpg  validation
tiger.195.jpg tiger.21.jpg  tiger.225.jpg tiger.26.jpg
tiger.196.jpg tiger.210.jpg tiger.226.jpg tiger.27.jpg
tiger.197.jpg tiger.211.jpg tiger.227.jpg tiger.28.jpg
tiger.198.jpg tiger.212.jpg tiger.228.jpg tiger.29.jpg


In [34]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import random

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 10
 
train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,3))
 
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=shuffle)

Found 1140 images belonging to 2 classes.


In [30]:
i = 0
nImages = 1140
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nImages:
        break
         
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

ValueError: could not broadcast input array from shape (10,2) into shape (10,3)

In [42]:
from keras.applications import VGG16
image_size = 224
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

ValueError: The input must have 3 channels; got `input_shape=(224, 224, 2)`

In [43]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2050      
Total params: 40,407,874
Trainable params: 40,407,874
Non-trainable params: 0
_________________________________________________________________


In [44]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1140 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [39]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')


Epoch 1/30


ValueError: Error when checking target: expected dense_2 to have shape (3,) but got array with shape (2,)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()